Data: https://www.kaggle.com/nasa/meteorite-landings

In [16]:
import numpy as np
from PIL import Image
import pandas as pd
import folium
import plotly.express as px
import plotly.graph_objects as go

Code to filter the meteorite data. Only needs to be run once.

In [17]:
meteorites = pd.read_csv("./data/meteorite-landings.csv", delimiter=",")
filtered_coords = (meteorites["Longitude"] >= -180) & (meteorites["Longitude"] <= 180) & ((meteorites["Longitude"] != 0) | (meteorites["Latitude"] != 0))
filtered_years = (meteorites["Year"] >= 860) & (meteorites["Year"] <= 2016)
filtered_mass = ((meteorites["Mass"] >= 100))
filtered_type = ((meteorites["Type"] == "Valid"))
filtered_meteorites = meteorites[filtered_coords & filtered_years & filtered_type]
filtered_meteorites.drop(["GeoLocation", "Type"], axis=1, inplace=True)
# filtered_meteorites["Mass"] = filtered_meteorites["Mass"].div(1000)
filtered_meteorites.to_csv("./data/cleansed-data.csv")
filtered_meteorites = filtered_meteorites.sort_values(by="Year", ascending=True)
filtered_meteorites

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Name,ID,Class,Mass,Fell,Year,Latitude,Longitude
703,Nogata,16988,L6,472.0,Fell,860.0,33.72500,130.75000
678,Narni,16914,Stone-uncl,NaN,Fell,920.0,42.51667,12.51667
278,Elbogen,7823,"Iron, IID",107000.0,Fell,1399.0,50.18333,12.73333
852,Rivolta de Bassi,22614,Stone-uncl,103.3,Fell,1490.0,45.48333,9.51667
283,Ensisheim,10039,LL6,127000.0,Fell,1491.0,47.86667,7.35000
...,...,...,...,...,...,...,...,...
38108,Tupelo,56551,EL6,280.0,Found,2012.0,34.24216,-88.77594
5371,Catalina 009,57173,CR2,5.2,Found,2012.0,-25.23333,-69.71667
30514,Northwest Africa 7496,56314,Eucrite-pmict,788.4,Found,2012.0,22.94781,-13.38527
17387,Ksar Ghilane 011,55606,L4,25.6,Found,2012.0,32.89100,9.91300


Code to plot the data

In [18]:
fig = px.scatter_geo(filtered_meteorites,lat='Latitude',lon='Longitude',hover_name="Name",color="Mass",projection="natural earth",animation_frame="Year",animation_group="Mass")
fig.update_layout(title = 'Meteorite Landings Across the World', title_x=.5)
fig.write_html("images/animation.html")
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed